In [ ]:
# default_exp metrics

# Custom losses and metrics

In [ ]:
# export
from drone_detector.imports import *
from fastai.learner import Metric
from fastai.torch_core import *
from fastai.metrics import *
from fastai.losses import BaseLoss
import sklearn.metrics as skm
import torch
import torch.nn.functional as F

/home/mayrajeo/miniconda3/envs/dronedetector/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# export
mk_class('ActivationType', **{o:o.lower() for o in ['No', 'Sigmoid', 'Softmax', 'BinarySoftmax']},
         doc="All possible activation classes for `AccumMetric")

In [ ]:
#export

def adjusted_R2Score(r2_score, n, k):
    "Calculates adjusted_R2Score based on r2_score, number of observations (n) and number of predictor variables(k)"
    return 1 - (((n-1)/(n-k-1)) * (1 - r2_score))


In [ ]:
#export

def _rrmse(inp, targ):
    "RMSE normalized with mean of the target"
    return torch.sqrt(F.mse_loss(inp, targ)) / targ.mean() * 100

rrmse = AccumMetric(_rrmse)
rrmse.__doc__ = "Target mean weighted rmse"

In [ ]:
#export
def _bias(inp, targ):
    "Average bias of predictions"
    inp, targ = flatten_check(inp, targ)
    return (inp - targ).sum() / len(targ)

bias = AccumMetric(_bias)
bias.__doc__ = "Average bias of predictions"

In [ ]:
#export
def _bias_pct(inp, targ):
    "Mean weighted bias"
    inp, targ = flatten_check(inp, targ)
    return 100 * ((inp-targ).sum()/len(targ)) / targ.mean()

bias_pct = AccumMetric(_bias_pct)
bias_pct.__doc__ = 'Mean weighted bias'

BigEarthNet metrics

In [ ]:
#export

def label_ranking_average_precision_score(sigmoid=True, sample_weight=None):
    """Label ranking average precision (LRAP) is the average over each ground truth label assigned to each sample, 
    of the ratio of true vs. total labels with lower score."""
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.label_ranking_average_precision_score, sample_weight=None, flatten=False, thresh=None, 
                         activation=activation)

In [ ]:
# export

def label_ranking_loss(sigmoid=True, sample_weight=None):
    """Compute the average number of label pairs that are incorrectly ordered given y_score 
    weighted by the size of the label set and the number of labels not in the label set."""
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.label_ranking_loss, sample_weight=None, flatten=False, thresh=None, 
                         activation=activation)

In [ ]:
# export

def _one_error(inp, targ):
    max_ranks = inp.argmax(axis=1)
    faults = 0
    for i in range_of(max_ranks):
        faults += targ[i,max_ranks[i]]
    return 1 - faults/len(max_ranks)
    
one_error = AccumMetric(_one_error, flatten=False)
one_error.__doc__ = "Rate for which the top ranked label is not among ground truth"

In [ ]:
# export

def coverage_error(sigmoid=True, sample_weight=None):
    """Compute how far we need to go through the ranked scores to cover all true labels. 
    The best value is equal to the average number of labels in y_true per sample."""
    
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.coverage_error, sample_weight=None, flatten=False, thresh=None, activation=activation)

In [ ]:
from fastai.learner import Learner
class TstLearner(Learner):
    def __init__(self,dls=None,model=None,**kwargs): self.pred,self.xb,self.yb = None,None,None

def compute_val(met, x1, x2):
    met.reset()
    vals = [0,6,15,20]
    learn = TstLearner()
    for i in range(3):
        learn.pred,learn.yb = x1[vals[i]:vals[i+1]],(x2[vals[i]:vals[i+1]],)
        met.accumulate(learn)
    return met.value

In [ ]:
lrap = label_ranking_average_precision_score()
lrl = label_ranking_loss()
cov = coverage_error()

In [ ]:
x_1 = torch.randn(10,10)
x_2 = torch.randint(2,(10,10))
x_1, torch.sigmoid(x_1), x_2

(tensor([[-0.7139, -1.1197,  1.3219, -0.8929, -0.0864, -1.0797, -1.1980, -1.1462,
           0.7752, -0.3072],
         [ 0.0160,  0.4422,  0.7312, -0.2738, -1.8118, -1.0051, -0.8846,  1.3535,
           0.4079,  1.8772],
         [-1.4298,  0.9177, -0.1495,  0.8309, -0.4060, -1.6582, -1.7383,  1.2100,
          -0.4028,  0.3403],
         [ 0.3563, -0.3367, -0.4472,  1.3416,  1.7071,  0.1720,  0.6515, -0.9848,
           1.3077, -0.0549],
         [-1.3293,  0.7442,  1.0438,  0.8183, -0.0366,  0.5047, -0.6310, -1.1718,
           0.1838,  0.2767],
         [-0.2497, -2.2051, -1.1139,  0.0787,  0.7320, -0.4195, -0.9570,  2.4100,
          -1.0918,  0.7197],
         [-0.5669,  1.1043,  0.7297, -0.3731,  1.2886, -0.6209, -0.7776,  1.3258,
           0.4056, -1.2151],
         [ 0.5525,  0.0561, -0.5651, -0.9875,  0.4355,  0.5439,  1.4667, -0.0607,
           0.5053, -0.5647],
         [ 0.0292, -2.3633, -0.6284,  0.9104, -0.5364, -0.8675, -0.3276, -0.5341,
           0.5528, -0.4312],
 

In [ ]:
compute_val(lrl, x_1, x_2)

0.4375

In [ ]:
compute_val(lrap, x_1, x_2)

0.6836692176870748

In [ ]:
compute_val(cov, x_1, x_2)

8.8

In [ ]:
one_error(x_1, x_2)

tensor(0.3000)